In [ ]:
import torch 
from torch.utils.data import DataLoader
from src import full_pcd_dataset
import importlib
importlib.reload(full_pcd_dataset)
import json
# Create dataset instance
dataset = full_pcd_dataset.FullPCDDataset("data/full_pcd_300000_samples_6d.npz")
with open("data/label_dict.json", "r") as f:
    label_dict = json.load(f)
num_classes = len(label_dict.keys())
# Split into train and validation (example: 80-20 split)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

# Create data loaders
batch_size = 128
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0,pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=0,pin_memory=True)
num_classes

In [ ]:
from src.model_updated import PointNet2Translation 
from torch.optim.lr_scheduler import StepLR
backbone_params = {"sa_mlps": [[32, 32, 32], [32, 64, 64]], "mlp":[256, 512, 1024], "downsample_points": [256, 64], "radii":[0.1, 0.2], "ks":[16, 32], "add_xyz":True}
model = PointNet2Translation(mlp = [1024, 512], backbone_params=backbone_params)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)
scheduler = StepLR(optimizer, step_size=5, gamma=0.5) 

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [ ]:
print(count_parameters(model))

In [ ]:
from src.train_position import train_model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

In [ ]:
train_model(model, train_loader, val_loader, optimizer, scheduler, device, epochs=100, directory="run_translation_new_data_larger_7")